#A Tale of a Reduction

> Objectives:
> * Compare operations taking place in different data containers
> * Compare sizes for these data containers
> * Help deciding when it is best to use a container or another

Let's suppose that we are going to need reductions a lot and we want to choose the best container for performing them.  First, let's start by activating our MemWatcher agent:

In [1]:
from ipython_memwatcher import MemWatcher
mw = MemWatcher()
mw.start_watching_memory()

In [1] used 0.035 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 29.016 MiB


and choose a different container for the data that we want to reduce, starting with a list:

##Regular lists

In [2]:
a = [i for i in range(1000*1000)]

In [2] used 31.254 MiB RAM in 0.140s, peaked 0.000 MiB above current, total RAM usage 60.270 MiB


Now, proceed with a simple reduction (sum):

In [3]:
t = %timeit -o sum(a)

100 loops, best of 3: 6.92 ms per loop
In [3] used 0.020 MiB RAM in 2.954s, peaked 0.000 MiB above current, total RAM usage 60.289 MiB


which, in MIPS (Mega-Instructions-Per-Second) is:

In [4]:
print("MIPS:", round(1e6 / t.best / 1e6, 1))

('MIPS:', 144.4)
In [4] used 0.008 MiB RAM in 0.003s, peaked 0.000 MiB above current, total RAM usage 60.297 MiB


Ok, so that seems fast, but we don't have other references to compare with.  In addition, a list is not the best kind of container in terms of space consumption.  So let's try now a container that seems quite optimal in terms of memory savings.

##Containers using the array module in Python

In [5]:
# Create an array of 'l'ong integers (8 bytes on 32-bit platforms)
import array
%load_ext memory_profiler

In [5] used 0.004 MiB RAM in 0.004s, peaked 0.000 MiB above current, total RAM usage 60.301 MiB


In [6]:
%memit arr = array.array('l', a)

peak memory: 75.49 MiB, increment: 15.19 MiB
In [6] used 15.316 MiB RAM in 0.362s, peaked 0.000 MiB above current, total RAM usage 75.617 MiB


7.7 ~ 15 MB vs 31 MB seems like a good deal, although sometimes Python is allocating more memory than necessary.  In fact, the array module seems to provide optimal containers from a memory consumption point of view:

In [7]:
# Size per element:
(mw.memory_delta * 2**20) / 1e6

16.060416

In [7] used 0.043 MiB RAM in 0.023s, peaked 0.000 MiB above current, total RAM usage 75.660 MiB


But how it performs during reductions?

In [10]:
t = %timeit -o sum(arr)

100 loops, best of 3: 11.6 ms per loop
In [10] used 0.004 MiB RAM in 4.851s, peaked 0.000 MiB above current, total RAM usage 75.695 MiB


In [11]:
print("MIPS:", round(1e6 / t.best / 1e6, 1))

('MIPS:', 86.5)
In [11] used 0.000 MiB RAM in 0.003s, peaked 0.000 MiB above current, total RAM usage 75.695 MiB


Well, that's a bit disappointing, as the array object performs about 2x slower than a regular array.  Not sure about the resons, but probably the array module is not getting too much attention performance-wise mainly because the NumPy existance.  Speaking of NumPy: here we go!

##NumPy

In [12]:
import numpy as np

In [12] used 8.840 MiB RAM in 0.090s, peaked 0.000 MiB above current, total RAM usage 84.535 MiB


In [13]:
na = np.array(a, dtype=np.int64)

In [13] used 7.645 MiB RAM in 0.102s, peaked 0.000 MiB above current, total RAM usage 92.180 MiB


We see that, with 8 bytes/element, NumPy is also an efficient container.

In [14]:
t = %timeit -o sum(na)

10 loops, best of 3: 83.2 ms per loop
In [14] used 0.012 MiB RAM in 3.615s, peaked 0.000 MiB above current, total RAM usage 92.191 MiB


In [15]:
print("MIPS:", round(1e6 / t.best / 1e6, 3))

('MIPS:', 12.013)
In [15] used 0.000 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 92.191 MiB


Oops, this is more than several times slower than the `array` module.  Why so?

**Answer:** NumPy has a lot of overhead in producing a Python integer for every element in the array.

*Hint:* Use internal methods (ufuncs) when possible.

In [16]:
t = %timeit -o na.sum()

1000 loops, best of 3: 543 µs per loop
In [16] used 0.133 MiB RAM in 2.351s, peaked 0.000 MiB above current, total RAM usage 92.324 MiB


In [17]:
print("MIPS:", round(1e6 / t.best / 1e6, 3))

('MIPS:', 1842.621)
In [17] used 0.000 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 92.324 MiB


This is about 100x the speed of sum() on a Python list and it is also pretty optimal in terms of both execution time and space consumed.

##Exercise

The speed in the above reduction is limited by memory speed, not CPU speed.  Could you provide a hint on the maximum speed that supports your laptop?

##Using compressed in-memory containers with bcolz

But let us suppose that we have really big data to process in our laptop and want to see if we can store our data in less space.  Enter compression:

In [18]:
import bcolz

In [18] used 27.184 MiB RAM in 0.436s, peaked 0.000 MiB above current, total RAM usage 119.508 MiB


In [20]:
ca = bcolz.carray(na)

In [20] used 0.277 MiB RAM in 0.010s, peaked 0.000 MiB above current, total RAM usage 128.008 MiB


In [21]:
print("mem_used:", mw.measurements.memory_delta)

('mem_used:', 0.27734375)
In [21] used 0.008 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 128.016 MiB


Ok, this time the amount of memory used seems much lower.  Let's see how much memory the container thinks it has:

In [23]:
ca

carray((1000000,), int64)
  nbytes: 7.63 MB; cbytes: 394.28 KB; ratio: 19.81
  cparams := cparams(clevel=5, shuffle=True, cname='blosclz')
[     0      1      2 ..., 999997 999998 999999]

In [23] used 0.004 MiB RAM in 0.007s, peaked 0.000 MiB above current, total RAM usage 128.055 MiB


In [24]:
t = %timeit -o ca.sum()
print("MIPS:", round(1e6 / t.best / 1e6, 3))

100 loops, best of 3: 1.98 ms per loop
('MIPS:', 505.763)
In [24] used 0.027 MiB RAM in 0.991s, peaked 0.000 MiB above current, total RAM usage 128.082 MiB


This is around 2~4x slower (depending on the machine) than a regular NumPy array, but the size of the array is an impressive 20x smaller.  Is compression the responsible of the  overhead?

## Using uncompressed containers with bcolz

In order to see if this is because of the compression overhead, let's use an uncompressed array:

In [25]:
cau = bcolz.carray(a, cparams=bcolz.cparams(clevel=0))

In [25] used 7.137 MiB RAM in 0.115s, peaked 0.000 MiB above current, total RAM usage 135.219 MiB


In [26]:
cau

carray((1000000,), int64)
  nbytes: 7.63 MB; cbytes: 7.75 MB; ratio: 0.98
  cparams := cparams(clevel=0, shuffle=True, cname='blosclz')
[     0      1      2 ..., 999997 999998 999999]

In [26] used 0.000 MiB RAM in 0.008s, peaked 0.000 MiB above current, total RAM usage 135.219 MiB


In [27]:
t = %timeit -o cau.sum()
print("MIPS:", round(1e6 / t.best / 1e6, 3))

100 loops, best of 3: 2.08 ms per loop
('MIPS:', 481.142)
In [27] used 0.008 MiB RAM in 1.028s, peaked 0.000 MiB above current, total RAM usage 135.227 MiB


As we can see, the times with an uncompressed `carray` are very close to a compressed one, so compressing is not the source of the overhead.

So, bcolz allows to use compressed in-memory data containers at the cost of some performance (compared with NumPy).  But the performance overhead is quite small, and sometimes you prefer to keep more data in-memory.

On another hand, in next notebooks we are going to see that bcolz can be competitive with NumPy performance wise in other scenarios.

### Exercise: Using bcolz in real scenarios

bcolz does not get good compression ratios only with synthetic data, but with real data too.  Be sure to check out this URL:

http://nbviewer.ipython.org/gist/alimanfoo/e93a532eb6bde311ea39/genotype_bitshuffle.ipynb

and let's discuss this specific case of bcolz usage in genomics:

* Which are the typical compression ratios for this case?

* Is there a difference in speed accessing data in compressed and non-compressed state (clevel=0)

* Which are the compressors achieving the best compression/speed ratio?